## StyleGAN - faces

#### Run the generative model called StyleGAN to generate fake faces.

You are going to use the original paper's implementation of StyleGAN. ( https://arxiv.org/pdf/1812.04948.pdf )

In [1]:
!git clone https://github.com/NVlabs/stylegan.git

Cloning into 'stylegan'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.


In [2]:
%tensorflow_version 1.x
# Import needed Python libraries
import os
import pickle
import warnings
import numpy as np
import PIL

from tensorflow.python.util import module_wrapper
module_wrapper._PER_MODULE_WARNING_LIMIT = 0

# Import the official StyleGAN repo
import stylegan
from stylegan.dnnlib import tflib
from stylegan import config

# Initialize TensorFlow
tflib.init_tf()

# Move into the StyleGAN directory, if you're not in it already
path = 'stylegan/'
if "stylegan" not in os.getcwd():
    os.chdir(path)

TensorFlow 1.x selected.


In [3]:
# Load pre-trained StyleGAN network
url = 'https://bitbucket.org/ezelikman/gans/downloads/karras2019stylegan-ffhq-1024x1024.pkl' # karras2019stylegan-ffhq-1024x1024.pkl
with stylegan.dnnlib.util.open_url(url, cache_dir=stylegan.config.cache_dir) as f:
  # You'll load 3 components, and use the last one Gs for sampling images.
  #   _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
  #   _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
  #   Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.
  _G, _D, Gs = pickle.load(f)

  print('StyleGAN package loaded successfully!')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
StyleGAN package loaded successfully!


In [4]:
#@title Generate faces with StyleGAN
#@markdown Double click here to see the code. After setting truncation, run the cells below to generate images. This adjusts the truncation, truncation trades off fidelity (quality) and diversity of the generated images - play with it!  

#@markdown `truncation`: The positive truncation value. 1 is low truncation (high diversity) and 0 is all truncation except for the mean (high quality/fidelity). A lower value increases fidelity and decreases diversity, and vice versa. These are trade-offs that you can play with.

Truncation = 0.2 #@param {type:"slider", min:0.1, max:1, step:0.1}

print(f'Truncation set to {Truncation}. \nNow run the cells below to generate images with this truncation value.')

Truncation set to 0.2. 
Now run the cells below to generate images with this truncation value.


In [5]:
# Set the random state. Nothing special about 24
rnd = np.random.RandomState(24)

print(f'Random state is set.')

Random state is set.


You'll default to 4 images for the run, which is called a batch. Feel free to generate more by changing this parameter, but note that very large batch sizes will cause the model to run out of memory.

In [6]:
batch_size = 4 #@param {type:"slider", min:1, max:10, step:1}

print(f'Batch size is {batch_size}...')

Batch size is 4...


Noise vectors make sure the generated images are randomly (stochastically) different, not all the same. Notice that there is a noise vector for each image in the batch. You can run this next cell as many times as you want to get new noise vectors, and as a result, new images!

In [7]:
input_shape = Gs.input_shape[1]
noise_vectors = rnd.randn(batch_size, input_shape)

print(f'There are {noise_vectors.shape[0]} noise vectors, each with {noise_vectors.shape[1]} random values between -{Truncation} and {Truncation}.')

There are 4 noise vectors, each with 512 random values between -0.2 and 0.2.


Run the model to generate the images. 

Notice that truncation and noise vectors are passed in. Don't worry too much about the other stuff - it's about output formats and adding additional randomness/diversity to the output.

In [8]:
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
images = Gs.run(noise_vectors, None, truncation_psi=Truncation, randomize_noise=False, output_transform=fmt)

print(f'Successfully sampled {batch_size} images from the model.')

Successfully sampled 4 images from the model.


Now you save and visualize the images. Feel free to regenerate the noise vectors above and run the cells afterwards to see new images. 

In [9]:
# Save the images
os.makedirs(config.result_dir, exist_ok=True)
png_filename = os.path.join(config.result_dir, 'stylegan-example.png')
if batch_size > 1:
  img = np.concatenate(images, axis=1)
else:
  img = images[0]
PIL.Image.fromarray(img, 'RGB').save(png_filename)

# Check the images out!
from IPython.display import Image
Image(png_filename, width=256*batch_size, height=256)

Output hidden; open in https://colab.research.google.com to view.

Here's another script to help familiarize you with what interpolation in GANs can look like—filling in the unknown between two objects. In this case, you are using face images where the GAN will fill in how one image can morph into another. You will generate random noise vectors for the start and end images and then create noise vectors for the the ones inbetween.

Again, feel free to mess with any of the code but here are some values you should try modifying:
  *   `rnd`: The random seed that generates the noise vectors. Changing this will change the noise vectors fed to the generator and, consequently, the images generated!
  *   `truncation`: The truncation value between 0 and 1. 1 is no truncation (high diversity) and 0 is all truncation except for the mean (high quality/fidelity). A lower value increases fidelity and decreases diversity, and vice versa. These are trade-offs that you can play with.
  *   `n_interpolations`: The number of images that you want generated. A 3 would mean there is one transition photo between the start and end photos. If you want to see an interpolation, this value should be greater than 2.

In [10]:
# Set the random seed that generates the noise vectors
rnd = np.random.RandomState(24)

# Set the truncation value for truncation trick sampling
truncation = 0.2

# Set the number of interpolations/number of images to generate
n_interpolation = 5

# Create a noise vector z for the start and end images (batch_size = 1 since they are single image): (batch_size, z_dim)
# And create noise for the interpolations inbetween
z_dim = Gs.input_shape[1]
first_noise = rnd.randn(1, z_dim)
second_noise = rnd.randn(1, z_dim)
percent_first_noise = np.linspace(0, 1, n_interpolation)[:, None]
interpolation_noise = first_noise * percent_first_noise + second_noise * (1 - percent_first_noise)

# Generate image by running (sampling) the generator
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True) # Specify the desired output format and shape
images = Gs.run(interpolation_noise,
                None,    # No labels/conditions because it is unconditional generation!
                truncation_psi=truncation, 
                randomize_noise=False,
                output_transform=fmt
                )

# Display images
if batch_size > 1:
  img = np.concatenate(images, axis=1) # Save all images in batch to a single image
else:
  img = images[0]
PIL.Image.fromarray(img, 'RGB')

Output hidden; open in https://colab.research.google.com to view.